# Swaps
- GetSwaps uses musd-pools-mezo


In [101]:
import pandas as pd
from dotenv import load_dotenv
import os
from mezo.currency_config import (
    POOLS_MAP, 
    POOL_TOKEN_PAIRS, 
    TOKENS_ID_MAP
)
from mezo.currency_utils import (
    format_pool_token_columns,
    add_pool_usd_conversions, 
    add_usd_conversions, 
    format_token_columns
)
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values
from mezo.clients import SubgraphClient
from decimal import Decimal


load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [2]:
SWAPS_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/musd-pools-mezo/1.1.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_SWAPS = """
query GetSwaps($skip: Int!) {
  swaps(
  first: 1000
  orderBy: timestamp_
  orderDirection: desc
  skip: $skip
  ) {
    timestamp_
    sender
    to
    contractId_
    amount0In
    amount0Out
    amount1In
    amount1Out
    transactionHash_
    block_number
  }
}
"""

In [3]:
# Instantiate subgraph
swaps = SubgraphClient(
    url = SWAPS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

In [4]:
# Fetch subgraph data
swap_data =  swaps.fetch_subgraph_data(GET_SWAPS, 'swaps')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
Fetching transactions with skip=5000...
Fetching transactions with skip=6000...
Fetching transactions with skip=7000...
Fetching transactions with skip=8000...
Fetching transactions with skip=9000...
Fetching transactions with skip=10000...
Fetching transactions with skip=11000...
Fetching transactions with skip=12000...
Fetching transactions with skip=13000...
Fetching transactions with skip=14000...
Fetching transactions with skip=15000...
Fetching transactions with skip=16000...
Fetching transactions with skip=17000...
Fetching transactions with skip=18000...
Fetching transactions with skip=19000...
Fetching transactions with skip=20000...
Fetching transactions with skip=21000...
Fetching transactions with skip=22000...
Fetching transactions with skip=23000...
Fetching transactions with sk

In [60]:
swaps_df = pd.DataFrame(swap_data)
display(swaps_df)

,timestamp_,sender,to,contractId_,amount0In,amount0Out,amount1In,amount1Out,transactionHash_,block_number
0,1759140353,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xb17ca51c9645eb0abdb9076f0ff3ea06d3ecff66,0x52e604c44417233b6ccedddc0d640a405caacefb,480000000000000,0,0,53554879407311079414,0x3378afa80b7f7e5e1a57c026202db6a45f72286da24e...,3526970
1,1759140233,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x97d85eeb7804e513bccec7b5175e9ad45a96d76b,0x52e604c44417233b6ccedddc0d640a405caacefb,270000000000000,0,0,30130824527459020870,0xbf2b8e06a890d1c424f06ce67a792630103f55c4fbaf...,3526937
2,1759137351,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x016040946d60a6cfc00fc1926a2ba50d537600cd,0x52e604c44417233b6ccedddc0d640a405caacefb,280000000000000,0,0,31251501493580282272,0xf5896772ab5d3e0485dd48823b724f4ab5647ab8f982...,3526138
3,1759136166,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x77c930da15354f32f7091abf6f37979bf153822c,0x52e604c44417233b6ccedddc0d640a405caacefb,250000000000000,0,0,27907188981431780388,0xb2bb22fc12b1f9e7b2d9653b1d2f2a12b2f3dc9f7fde...,3525809
4,1759135645,0xd4602cd70960a06c382c2ca250e34b54841c6365,0xd4602cd70960a06c382c2ca250e34b54841c6365,0x52e604c44417233b6ccedddc0d640a405caacefb,0,1202436208263343,135000000000000000000,0,0x0cff46855e98d196a90b486e205110bafef305562c63...,3525665
...,...,...,...,...,...,...,...,...,...,...
40066,1752013895,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,250000000000000,0,0,28463880024065974068,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,1555293
40067,1751924558,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x52e604c44417233b6ccedddc0d640a405caacefb,0,157198903110751,18910000000000000000,0,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,1530335
40068,1751924405,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,2010400,0,0,2009989838895249558,0x3afbf42ee8974285284f7bc0a5f621a5022cee2f0a7e...,1530292
40069,1750952336,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x10906a9e9215939561597b4c8e4b98f93c02031a,0,999896001631977073,1000000,0,0x14869a1d4d2fd0a499d95c70d104132e1d6bc31372c7...,1264305


In [108]:
from mezo.currency_config import MEZO_ASSET_NAMES_MAP

df = swaps_df.copy()

df['pool'] = df['contractId_'].map(POOLS_MAP)

In [109]:
# df = format_datetimes(swaps_df, date_columns=['timestamp_'])
df = format_pool_token_columns(df, 'contractId_', POOL_TOKEN_PAIRS)

token_columns = ['token0', 'token1']
for col in token_columns:
    df[col] = df[col].replace(MEZO_ASSET_NAMES_MAP)

df.tail(10)

,timestamp_,sender,to,contractId_,amount0In,amount0Out,amount1In,amount1Out,transactionHash_,block_number,pool,token0,token1
40061,2025-07-09,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0.000000,2.009506,2.0099,0.000000,0xbe7e90150f1ad0a4b3bf58bfb83f73b9c04a424072d2...,1565692,musdc_musd_pool,USDC,MUSD
40062,2025-07-09,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,0.001091,0.000000,0.0000,118.939918,0x6676182aeb2776b433a882e70c3ad7613b2c92a6f339...,1564959,btc_musd_pool,BTC,MUSD
40063,2025-07-09,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,0.000000,0.000034,3.7000,0.000000,0x55cfb191f022b6e919cbec9b532f7a4240f2d039f4d6...,1564766,btc_musd_pool,BTC,MUSD
40064,2025-07-09,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,0.000090,0.000000,0.0000,9.966086,0xde94e99f474c3bd0770a366e2af076f0f0ed7a482988...,1559712,btc_musd_pool,BTC,MUSD
40065,2025-07-09,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,0.000000,0.000148,16.0000,0.000000,0xb4384d176ae0a2e426da9aaf3aa07d67fd05912aca70...,1559679,btc_musd_pool,BTC,MUSD
40066,2025-07-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,0.000250,0.000000,0.0000,28.463880,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,1555293,btc_musd_pool,BTC,MUSD
40067,2025-07-07,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x52e604c44417233b6ccedddc0d640a405caacefb,0.000000,0.000157,18.9100,0.000000,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,1530335,btc_musd_pool,BTC,MUSD
40068,2025-07-07,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,2.010400,0.000000,0.0000,2.009990,0x3afbf42ee8974285284f7bc0a5f621a5022cee2f0a7e...,1530292,musdc_musd_pool,USDC,MUSD
40069,2025-06-26,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x10906a9e9215939561597b4c8e4b98f93c02031a,0.000000,0.999896,1.0000,0.000000,0x14869a1d4d2fd0a499d95c70d104132e1d6bc31372c7...,1264305,musd_musdt_pool,MUSD,USDT
40070,2025-05-28,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,0.000000,0.000009,1.0000,0.000000,0x914bbd24eba8cc9a36a0c82ecf91d1f77eb6a6d04847...,558263,btc_musd_pool,BTC,MUSD


In [ ]:
# Add USD conversions
df = add_usd_conversions(df, 'token0', TOKENS_ID_MAP, ['amount0In', 'amount0Out'])
df = df.drop(columns=['index', 'usd'])
df = add_usd_conversions(df, 'token1', TOKENS_ID_MAP, ['amount1In', 'amount1Out'])
df = df.drop(columns=['index', 'usd'])

In [112]:
df[['timestamp_', 'pool', 'token0', 'token1', 
    'amount0In', 'amount0Out', 
    'amount0In_usd', 'amount0Out_usd', 
    'amount1In', 'amount1Out', 
    'amount1In_usd', 'amount1Out_usd']]

,timestamp_,pool,token0,token1,amount0In,amount0Out,amount0In_usd,amount0Out_usd,amount1In,amount1Out,amount1In_usd,amount1Out_usd
0,2025-09-29,btc_musd_pool,BTC,MUSD,0.00048,0.000000,54.270240,0.000000,0.00,53.554879,0.00,53.554879
1,2025-09-29,btc_musd_pool,BTC,MUSD,0.00027,0.000000,30.527010,0.000000,0.00,30.130825,0.00,30.130825
2,2025-09-29,btc_musd_pool,BTC,MUSD,0.00028,0.000000,31.657640,0.000000,0.00,31.251501,0.00,31.251501
3,2025-09-29,btc_musd_pool,BTC,MUSD,0.00025,0.000000,28.265750,0.000000,0.00,27.907189,0.00,27.907189
4,2025-09-29,btc_musd_pool,BTC,MUSD,0.00000,0.001202,0.000000,135.951045,135.00,0.000000,135.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
40066,2025-07-08,btc_musd_pool,BTC,MUSD,0.00025,0.000000,28.265750,0.000000,0.00,28.463880,0.00,28.463880
40067,2025-07-07,btc_musd_pool,BTC,MUSD,0.00000,0.000157,0.000000,17.773380,18.91,0.000000,18.91,0.000000
40068,2025-07-07,musdc_musd_pool,USDC,MUSD,2.01040,0.000000,2.009624,0.000000,0.00,2.009990,0.00,2.009990
40069,2025-06-26,musd_musdt_pool,MUSD,USDT,0.00000,0.999896,0.000000,0.999896,1.00,0.000000,1.00,0.000000


In [114]:
short = df[['timestamp_', 'pool', 'token0', 'token1', 
    'amount0In_usd', 'amount1In_usd', 'amount0Out_usd',
    'amount1Out_usd', 'transactionHash_', 'to']]

In [115]:
def get_volume_for_row(row):
    """Determine which volume to use based on token types."""
    volatiles = ['SolvBTC', 'xSolvBTC']
    stables = ['USDC', 'USDT', 'upMUSD']
    
    token0 = row['token0']
    token1 = row['token1']
    vol0 = row['amount0In_usd']
    vol1 = row['amount1In_usd']
    
    # priority 1: MUSD
    if token0 == 'MUSD':
        return vol0
    elif token1 == 'MUSD':
        return vol1
    # priority 2: other stables
    elif token0 in stables and token1 not in stables:
        return vol0
    elif token1 in stables and token0 not in stables:
        return vol1
    # priority 3: take BTC if pair is another volatile asset
    elif token0 == 'BTC' and token1 in volatiles:
        return vol0
    elif token1 == 'BTC' and token0 in volatiles:
        return vol1
    else:
        return max(vol0, vol1)

In [ ]:
short = short.sort_values(['pool', 'timestamp_'])
short['volume'] = short.apply(get_volume_for_row, axis=1)
short.tail(30)

,timestamp_,pool,token0,token1,amount0In_usd,amount1In_usd,amount0Out_usd,amount1Out_usd,transactionHash_,to,volume
596,2025-09-25,musdc_musdt_pool,USDC,USDT,0.000000,404.540780,403.237964,0.0,0x20b23692452a59a074e6c12daa6beb1bb403158f7024...,0xc65bb13d8fa8b70b3d112bacdbacd7892b844808,404.540780
619,2025-09-25,musdc_musdt_pool,USDC,USDT,0.000000,134.964566,135.205716,0.0,0xb4cc03ed540da4ecf0eee57a3263351f74548959b65b...,0xc65bb13d8fa8b70b3d112bacdbacd7892b844808,134.964566
626,2025-09-25,musdc_musdt_pool,USDC,USDT,0.000000,44.990614,45.309918,0.0,0x629556ca56818ad36ec43be707f3b0a4cf375dcd74e4...,0xc65bb13d8fa8b70b3d112bacdbacd7892b844808,44.990614
629,2025-09-25,musdc_musdt_pool,USDC,USDT,0.000000,44.990927,45.530986,0.0,0x8d5719d4e9a268a8745a1647f7abbc9f165a0ff5f0f9...,0xc65bb13d8fa8b70b3d112bacdbacd7892b844808,44.990927
237,2025-09-27,musdc_musdt_pool,USDC,USDT,134.974886,0.000000,134.947890,0.0,0xdb466b0c1aa03c25d3cc85dde020f59800feb6583bea...,0xc65bb13d8fa8b70b3d112bacdbacd7892b844808,134.974886


In [119]:
short.head(30)

,timestamp_,pool,token0,token1,amount0In_usd,amount1In_usd,amount0Out_usd,amount1Out_usd,transactionHash_,to,volume
0,2025-05-28,btc_musd_pool,BTC,MUSD,0.000000,1.000000,1.035885,0.000000,0x914bbd24eba8cc9a36a0c82ecf91d1f77eb6a6d04847...,0x52d033e64774f31a8b7562806aa5782cbbd29382,1.000000
1,2025-07-07,btc_musd_pool,BTC,MUSD,0.000000,18.910000,17.773380,0.000000,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,0x6e80164ea60673d64d5d6228beb684a1274bb017,18.910000
2,2025-07-08,btc_musd_pool,BTC,MUSD,28.265750,0.000000,0.000000,28.463880,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,0x52d033e64774f31a8b7562806aa5782cbbd29382,0.000000
3,2025-07-09,btc_musd_pool,BTC,MUSD,0.000000,600.000000,611.528812,0.000000,0xb41095a7e3b4d1be82dc868449da8eb5a75ef95b694e...,0x52d033e64774f31a8b7562806aa5782cbbd29382,600.000000
4,2025-07-09,btc_musd_pool,BTC,MUSD,1.040091,0.000000,0.000000,0.999478,0xb5d577f1d438b5d5d9ea0f71d551b704fafa67ceeccb...,0xd19b598712413e69b48f70c5ea16286cf8dfd632,0.000000
5,2025-07-09,btc_musd_pool,BTC,MUSD,123.351733,0.000000,0.000000,118.939918,0x6676182aeb2776b433a882e70c3ad7613b2c92a6f339...,0x52d033e64774f31a8b7562806aa5782cbbd29382,0.000000
6,2025-07-09,btc_musd_pool,BTC,MUSD,0.000000,3.700000,3.872413,0.000000,0x55cfb191f022b6e919cbec9b532f7a4240f2d039f4d6...,0x52d033e64774f31a8b7562806aa5782cbbd29382,3.700000
7,2025-07-09,btc_musd_pool,BTC,MUSD,10.175670,0.000000,0.000000,9.966086,0xde94e99f474c3bd0770a366e2af076f0f0ed7a482988...,0x52d033e64774f31a8b7562806aa5782cbbd29382,0.000000
8,2025-07-09,btc_musd_pool,BTC,MUSD,0.000000,16.000000,16.746486,0.000000,0xb4384d176ae0a2e426da9aaf3aa07d67fd05912aca70...,0x52d033e64774f31a8b7562806aa5782cbbd29382,16.000000
9,2025-07-10,btc_musd_pool,BTC,MUSD,0.000000,187.530000,187.932938,0.000000,0x93acc97e3dcdd1e70f2b263c5c116ad588f82926da6b...,0x52d033e64774f31a8b7562806aa5782cbbd29382,187.530000


In [117]:
short = short.reset_index(drop=True)

In [120]:
short['total_inflow'] = short['amount0In_usd'] + short['amount1In_usd']
short['total_outflow'] = short['amount0Out_usd'] + short['amount1Out_usd']

In [ ]:
# daily
daily_swaps_by_pool = short.groupby(['timestamp_', 'pool']).agg(
    total_swaps = ('count', 'sum'),
    users = ('to', lambda x: x.nunique()),
    volume = ('volume', 'sum')
    inflow_0 = ('amount0In_usd', 'sum')
    inflow_1 = ('amount1In_usd', 'sum')
    inflow = ('total_inflow', 'sum'),
    outflow = ('total_outflow', 'sum'),
).reset_index()

In [ ]:
# daily
daily_swaps = swaps_df.groupby(['timestamp_']).agg(
    total_swaps = ('count', 'sum'),
    users = ('to', lambda x: x.nunique()),
    first_asset_in = ('amount0In', 'sum'),
    first_asset_out = ('amount0Out', 'sum'),
    musd_in = ('amount1In', 'sum'),
    musd_out = ('amount1Out', 'sum')
).reset_index()

In [ ]:
daily_swaps_by_pool = daily_swaps_by_pool.pivot(
    index='timestamp_', columns='pool'
).fillna(0)

In [ ]:
daily_swaps_by_pool.columns = [
'_'.join(col).strip() for col in daily_swaps_by_pool.columns.values
]

daily_swaps_by_pool = daily_swaps_by_pool.reset_index()

# Fees processing

In [28]:
from mezo.queries import MUSDQueries

raw_fees_data = SubgraphClient.get_subgraph_data(
    SubgraphClient.SWAPS_SUBGRAPH, 
    MUSDQueries.GET_FEES_FOR_SWAPS,
    'fees'
)

🔍 Trying fees query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
Fetching transactions with skip=5000...
Fetching transactions with skip=6000...
Fetching transactions with skip=7000...
Fetching transactions with skip=8000...
Fetching transactions with skip=9000...
Fetching transactions with skip=10000...
Fetching transactions with skip=11000...
Fetching transactions with skip=12000...
Fetching transactions with skip=13000...
Fetching transactions with skip=14000...
Fetching transactions with skip=15000...
Fetching transactions with skip=16000...
Fetching transactions with skip=17000...
Fetching transactions with skip=18000...
Fetching transactions with skip=19000...
Fetching transactions with skip=20000...
Fetching transactions with skip=21000...
Fetching transactions with skip=22000...
Fetching transactions with skip=23000...
Fetchi

In [141]:
swaps_df['to'].unique()

array(['0xb17ca51c9645eb0abdb9076f0ff3ea06d3ecff66',
       '0x97d85eeb7804e513bccec7b5175e9ad45a96d76b',
       '0x016040946d60a6cfc00fc1926a2ba50d537600cd', ...,
       '0x7b9f56d185d48d56386352114fc88b6ddcd3edf8',
       '0x8c866cda92443ef27f7ab0f7566d56ddda8f518a',
       '0xd19b598712413e69b48f70c5ea16286cf8dfd632'],
      shape=(9455,), dtype=object)

In [29]:
fees_df = pd.DataFrame(raw_fees_data)

In [139]:
fees_df['sender'].unique()

array(['0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706',
       '0xd4602cd70960a06c382c2ca250e34b54841c6365',
       '0xc65bb13d8fa8b70b3d112bacdbacd7892b844808',
       '0xaba85109ab63a3c1600e73550643a2193b9398a5',
       '0x6037c8fd5a32fa44ed358ef96d94bd323d5b5e90',
       '0x84e6757e9425bb747c0f33ad8b7aafea5b2d26ec',
       '0x769e6fff89671af73d907c64c4769b04033c4885'], dtype=object)

In [125]:
df2 = fees_df.copy()

df2['pool'] = df2['contractId_'].map(POOLS_MAP)
df2 = format_datetimes(df2, date_columns=['timestamp_'])
df2 = format_pool_token_columns(df2, 'contractId_', POOL_TOKEN_PAIRS)
df2 = add_pool_usd_conversions(df2, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)
df2['count'] = 1

df2.head(10)

,timestamp_,sender,amount0,amount1,contractId_,transactionHash_,pool,token0,token1,token0_index,token1_index,token0_usd_rate,token1_usd_rate,amount0_usd,amount1_usd,count
0,2025-09-29,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,1.440000e-06,0.000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x3378afa80b7f7e5e1a57c026202db6a45f72286da24e...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112858.000000,1.0,0.162516,0.000,1
1,2025-09-29,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,8.100000e-07,0.000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xbf2b8e06a890d1c424f06ce67a792630103f55c4fbaf...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112858.000000,1.0,0.091415,0.000,1
2,2025-09-29,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,8.400000e-07,0.000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xf5896772ab5d3e0485dd48823b724f4ab5647ab8f982...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112858.000000,1.0,0.094801,0.000,1
3,2025-09-29,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,7.500000e-07,0.000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xb2bb22fc12b1f9e7b2d9653b1d2f2a12b2f3dc9f7fde...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112858.000000,1.0,0.084643,0.000,1
4,2025-09-29,0xd4602cd70960a06c382c2ca250e34b54841c6365,0.000000e+00,0.405,0x52e604c44417233b6ccedddc0d640a405caacefb,0x0cff46855e98d196a90b486e205110bafef305562c63...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112858.000000,1.0,0.000000,0.405,1
5,2025-09-29,0xd4602cd70960a06c382c2ca250e34b54841c6365,0.000000e+00,0.405,0x52e604c44417233b6ccedddc0d640a405caacefb,0x4b22dfcd17718625e00bbd1e46c76ee024512acf3f7e...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112858.000000,1.0,0.000000,0.405,1
6,2025-09-29,0xd4602cd70960a06c382c2ca250e34b54841c6365,0.000000e+00,0.009,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xcd1494410bf39a73674c5ce11abf33e978d3b2fe8e8e...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999643,1.0,0.000000,0.009,1
7,2025-09-29,0xd4602cd70960a06c382c2ca250e34b54841c6365,0.000000e+00,1.215,0x52e604c44417233b6ccedddc0d640a405caacefb,0x1bad168b6355934c40e2624b0b0dfbeb836ea3c20bd8...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112858.000000,1.0,0.000000,1.215,1
8,2025-09-29,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,1.800000e-06,0.000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x1ae776284bfea4ebcb51858a44fc58eca964f3daa99d...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112858.000000,1.0,0.203144,0.000,1
9,2025-09-29,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000000e+00,0.003,0x52e604c44417233b6ccedddc0d640a405caacefb,0x91f029c21f2edf662eb2099d741892106979df280ee2...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112858.000000,1.0,0.000000,0.003,1


In [127]:
short.columns

Index(['timestamp_', 'pool', 'token0', 'token1', 'amount0In_usd',
       'amount1In_usd', 'amount0Out_usd', 'amount1Out_usd', 'transactionHash_',
       'to', 'volume', 'total_inflow', 'total_outflow'],
      dtype='object')

In [134]:
short.head(25)

,timestamp_,pool,token0,token1,amount0In_usd,amount1In_usd,amount0Out_usd,amount1Out_usd,transactionHash_,to,volume,total_inflow,total_outflow
0,2025-05-28,btc_musd_pool,BTC,MUSD,0.000000,1.000000,1.035885,0.000000,0x914bbd24eba8cc9a36a0c82ecf91d1f77eb6a6d04847...,0x52d033e64774f31a8b7562806aa5782cbbd29382,1.000000,1.000000,1.035885
1,2025-07-07,btc_musd_pool,BTC,MUSD,0.000000,18.910000,17.773380,0.000000,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,0x6e80164ea60673d64d5d6228beb684a1274bb017,18.910000,18.910000,17.773380
2,2025-07-08,btc_musd_pool,BTC,MUSD,28.265750,0.000000,0.000000,28.463880,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,0x52d033e64774f31a8b7562806aa5782cbbd29382,0.000000,28.265750,28.463880
3,2025-07-09,btc_musd_pool,BTC,MUSD,0.000000,600.000000,611.528812,0.000000,0xb41095a7e3b4d1be82dc868449da8eb5a75ef95b694e...,0x52d033e64774f31a8b7562806aa5782cbbd29382,600.000000,600.000000,611.528812
4,2025-07-09,btc_musd_pool,BTC,MUSD,1.040091,0.000000,0.000000,0.999478,0xb5d577f1d438b5d5d9ea0f71d551b704fafa67ceeccb...,0xd19b598712413e69b48f70c5ea16286cf8dfd632,0.000000,1.040091,0.999478
5,2025-07-09,btc_musd_pool,BTC,MUSD,123.351733,0.000000,0.000000,118.939918,0x6676182aeb2776b433a882e70c3ad7613b2c92a6f339...,0x52d033e64774f31a8b7562806aa5782cbbd29382,0.000000,123.351733,118.939918
6,2025-07-09,btc_musd_pool,BTC,MUSD,0.000000,3.700000,3.872413,0.000000,0x55cfb191f022b6e919cbec9b532f7a4240f2d039f4d6...,0x52d033e64774f31a8b7562806aa5782cbbd29382,3.700000,3.700000,3.872413
7,2025-07-09,btc_musd_pool,BTC,MUSD,10.175670,0.000000,0.000000,9.966086,0xde94e99f474c3bd0770a366e2af076f0f0ed7a482988...,0x52d033e64774f31a8b7562806aa5782cbbd29382,0.000000,10.175670,9.966086
8,2025-07-09,btc_musd_pool,BTC,MUSD,0.000000,16.000000,16.746486,0.000000,0xb4384d176ae0a2e426da9aaf3aa07d67fd05912aca70...,0x52d033e64774f31a8b7562806aa5782cbbd29382,16.000000,16.000000,16.746486
9,2025-07-10,btc_musd_pool,BTC,MUSD,0.000000,187.530000,187.932938,0.000000,0x93acc97e3dcdd1e70f2b263c5c116ad588f82926da6b...,0x52d033e64774f31a8b7562806aa5782cbbd29382,187.530000,187.530000,187.932938


In [128]:
short2 = df2[[
    'timestamp_', 'pool', 'token0', 'token1',
    'amount0_usd', 'amount1_usd', 'sender', 'transactionHash_'
    ]]

In [129]:
swaps_with_fees = pd.merge(
    short, 
    short2, 
    how='left', 
    on='transactionHash_',
    suffixes=('', '_fee')
)

In [142]:
# Select and rename columns for clarity
col_map = {
    'timestamp_': 'timestamp',
    'to': 'user',
    'pool': 'pool',
    'token0': 'token0',
    'token1': 'token1',
    'amount0In_usd': 'amount0In_usd',
    'amount1In_usd': 'amount1In_usd',
    'amount0Out_usd': 'amount0Out_usd',
    'amount1Out_usd': 'amount1Out_usd',
    'amount0_usd': 'fee0_usd',
    'amount1_usd': 'fee1_usd',
    'transactionHash_': 'transactionHash_'
}

In [143]:
int_swaps_with_fees = swaps_with_fees.rename(columns=col_map)

In [144]:
int_swaps_with_fees.columns

Index(['timestamp', 'pool', 'token0', 'token1', 'amount0In_usd',
       'amount1In_usd', 'amount0Out_usd', 'amount1Out_usd', 'transactionHash_',
       'user', 'volume', 'total_inflow', 'total_outflow', 'timestamp__fee',
       'pool_fee', 'token0_fee', 'token1_fee', 'fee0_usd', 'fee1_usd',
       'sender'],
      dtype='object')